# 12.1 Optimization: Simulated Annealing 


#### This class:

* Simulated Annealing
    - Travelling salesman problem
    
#### Before we finish
* Please do your [Course experience survey](https://ces.uvic.ca/blue)

## Optimization
$f(x) = x^2 - \cos(4 \pi x)$


### [Simulated annealing](https://en.wikipedia.org/wiki/Simulated_annealing)

* Is a [probabilistic technique](https://en.wikipedia.org/wiki/Probabilistic_algorithm) for approximating the [global optimum](https://en.wikipedia.org/wiki/Global_optimum) of a given [function](https://en.wikipedia.org/wiki/Function_(mathematics)). 
* Inspiration come from [annealing in metallurgy](https://en.wikipedia.org/wiki/Annealing_(metallurgy)), a technique involving heating and controlled cooling of a material to  increase the size of its crystals and reduce their defects
* The notion of slow cooling implemented in the simulated annealing  algorithm is interpreted as a slow decrease in the probability of  accepting worse solutions as the solution space is explored
* Simulated annealing algorithms work as follows
  * The temperature  progressively decreases from an initial positive value to zero. 
  * At each  time step, the algorithm 
    * randomly selects a solution close to the  current one, 
    * measures its quality, and 
    * moves to it according to the  temperature-dependent probabilities of selecting better or worse  solutions
* The neighbours of a state
  * Evaluating the neighbours of a state of the problem, which are new states produced through conservatively altering a given state
  * The well-defined way in which the states are altered to produce neighbouring states is called a _move_
  * Moves usually result in minimal alterations of the last state
* Acceptance probablilities
  * The probability of making the [transition](https://en.wikipedia.org/wiki/State_transition) from the current state $s$ to a candidate new state $s'$  is specified by an acceptance probability function $P(e,e',T)$  that depends on the energies $e = E(s)$ and $e' = E(s')$ of the two states, and on a global time-varying parameter $T$ called the *temperature*.
  * States with a smaller energy are better than those with a greater energy.  
  * When $T$ tends to zero, the probability $P(e,e',T)$ must tend to zero if $e'>e$ and to a positive value otherwise.
  * The $P$ function is usually chosen so that the probability of accepting a move decreases when the difference $e'-e$ increases
* The annealing schedule
  * The algorithm starts initially with $T$ set to a high value (or infinity), and then it is decreased at each step following some *annealing schedule*—which may be specified by the user, but must end with $T=0$ towards the end of the allotted time budget.  

#### Pseudocode

- Let $s = s_0$
- For $k = 0$ through $k_\mathrm{max}$ (exclusive):
  - $T$ ← $temperature( (k+1)/k_\mathrm{max} )$
  - Pick a random neighbour, $s_\mathrm{new}$ ← $neighbour(s)$
  - If  $P(E(s), E(s_\mathrm{new}), T) ≥ random(0, 1)$
    - $s$ ← $s_\mathrm{new}$
- Output: the final state $s$



#### What we need 

* state space with states $s$
* energy function $E(s)$ (goal)
* a `neighbour` function to produce new $s$
* the acceptance probability function `P()`
* the annealing schedule `temperature` and initial temperature $T_0$

#### Generic APF

For the Acceptance Probability Function we can generally use

$P(e,e',T) = 1$ if  $e'<e$ and $P(e,e',T) = exp(-(e'-e)/T)$ otherwise.



#### Travelling salesman problem

In the [travelling salesman problem](https://en.wikipedia.org/wiki/Travelling_salesman_problem) each state is typically defined as a [permutation](https://en.wikipedia.org/wiki/Permutation) of the cities to be visited, and its neighbours are the set of  permutations produced by reversing the order of any two successive  cities.

In [1]:
%pylab ipympl

Populating the interactive namespace from numpy and matplotlib


In [18]:
# cities, initial state
ncity = 5
s = array([random.randint(2*ncity+1) for i in range(2*ncity)],dtype='int')
s = s.reshape(ncity,2)

In [2]:
ncity = 4

In [17]:
array([random.randint(2*ncity) for i in range(2*ncity)]).reshape(ncity,2)

array([[3, 6],
       [6, 4],
       [3, 5],
       [0, 4]])

In [20]:
s

array([[7, 4],
       [3, 9],
       [0, 7],
       [8, 9],
       [8, 7]])

In [19]:
s.T[0]

array([7, 3, 0, 8, 8])

In [43]:
ifig=11;close(ifig);fig=figure(ifig)
size=5
fig.canvas.layout.height = str(size)+'in'   # This is a hack to prevent ipympl
fig.canvas.layout.width  = str(size)+'in'   # to adjust horizontal figure size

plot(s.T[0],s.T[1],'o-')
plot(sp.T[0],sp.T[1],'--')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [22]:
s

array([[7, 4],
       [3, 9],
       [0, 7],
       [8, 9],
       [8, 7]])

In [173]:
s.T

array([[ 9, 10,  7,  2,  7],
       [ 8,  7,  1,  7,  9]])

### Energy function

In [45]:
s.T

array([[7, 3, 0, 8, 8],
       [4, 9, 7, 9, 7]])

In [47]:
diff(s.T)**2

array([[16,  9, 64,  0],
       [25,  4,  4,  4]])

In [51]:
sTds = diff(s.T)**2
sTds

array([[16,  9, 64,  0],
       [25,  4,  4,  4]])

In [53]:
sqrt(sTds.sum(axis=0))

array([6.40312424, 3.60555128, 8.24621125, 2.        ])

In [54]:
sqrt(sTds.sum(axis=0)).sum()

20.25488676413216

In [55]:
energy = lambda s: sqrt(sum(diff(s.T)**2,axis=0)).sum()

In [56]:
energy(s)

20.25488676413216

In [57]:
energy(sp)

27.64933548866817

### Neighbour (or move)

In [24]:
s

array([[7, 4],
       [3, 9],
       [0, 7],
       [8, 9],
       [8, 7]])

In [26]:
iswap = random.randint(ncity-1)
iswap

3

In [27]:
arange(1,5,1)

array([1, 2, 3, 4])

In [29]:
arange(1,3,1)

array([1, 2])

In [31]:
arange(1,5,1)[array((0,3))]

array([1, 4])

index array

In [35]:
ind_arr = array((iswap,iswap+1))
ind_arr

array([3, 4])

In [36]:
ind_arr[::-1]

array([4, 3])

In [34]:
s[ind_arr]

array([[8, 9],
       [8, 7]])

In [37]:
s[ind_arr[::-1]]

array([[8, 7],
       [8, 9]])

In [38]:
def neighbour(s):
    sp = copy(s)
    ncity = len(s)
    iswap = random.randint(ncity-1)  # random number of left cities, swap with right neighbour
    ind_arr=array((iswap,iswap+1))
    sp[ind_arr] = s[ind_arr[::-1]]
    return sp,iswap

In [39]:
s

array([[7, 4],
       [3, 9],
       [0, 7],
       [8, 9],
       [8, 7]])

In [40]:
sp,iswap = neighbour(s)
print(iswap)
# plot(sp.T[0],sp.T[1],':')

2


In [41]:
sp

array([[7, 4],
       [3, 9],
       [8, 9],
       [0, 7],
       [8, 7]])

In [295]:
print(energy(sp),energy(s))

24.94427190999916 24.224381799279676


In [292]:
s = copy(sp)

### APF
Acceptance probability function

In [ ]:
ifig=12;close(ifig);figure(ifig)


In [63]:
def P(sp,s,T):
    ep,e = energy(sp),energy(s)
    if ep <= e:
        P = 1
    else:
        P = exp(-(ep-e)/T)
    return P

In [64]:
Tgrid = arange(20,0,-0.1)

In [387]:
Tgrid0 = copy(10*exp(-1*grid))

In [73]:
# explore temperature cooling schedule
ifig=37;close(ifig);fig=figure(ifig)
grid = linspace(0,15,100)
Tgrid = 10*exp(-1*grid)
plot(Tgrid)
# plot(Tgrid,'--')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [67]:
ifig=12;close(ifig);figure(ifig)
plot(Tgrid, P(sp,s,Tgrid),'--')
# plot(Tgrid,exp(-(0.21)/Tgrid))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [74]:
ncity = 8
Tgrid = arange(50,0,-0.1)
s = array([random.randint(2*ncity+1) for i in range(2*ncity)],dtype='int')
s = s.reshape(ncity,2)
sini = copy(s)
print("{:10.3f}".format(energy(s)))
ef = energy(sini)
sf = copy(sini)
for T in Tgrid:
    sp,iswap = neighbour(s)
    PP = P(sp,s,T)
    RR = random.random()
    print("{:10.3f} {:10.3f} {:8.3f} {:8.3f} {:8.3f}".format(energy(s),energy(sp),PP,RR,T))
    if PP >= RR: 
        s = copy(sp)
        if energy(s) < energy(sf): sf = copy(s)
        
print("{:10.3f} {:10.3f} {:8.3f} {:8.3f} {:8.3f}".format(energy(s),energy(sp),PP,RR,T))
print("{:10.3f}".format(energy(sf)))

    67.997
    67.997     66.353    1.000    0.872   50.000
    66.353     67.997    0.968    0.108   49.900
    67.997     73.640    0.893    0.852   49.800
    73.640     53.487    1.000    0.044   49.700
    53.487     74.592    0.653    0.486   49.600
    74.592     74.128    1.000    0.349   49.500
    74.128     74.592    0.991    0.200   49.400
    74.592     53.487    1.000    0.414   49.300
    53.487     51.163    1.000    0.033   49.200
    51.163     52.649    0.970    0.061   49.100
    52.649     48.443    1.000    0.405   49.000
    48.443     46.200    1.000    0.196   48.900
    46.200     67.305    0.649    0.578   48.800
    67.305     67.925    0.987    0.008   48.700
    67.925     69.362    0.971    0.641   48.600
    69.362     68.638    1.000    0.439   48.500
    68.638     66.414    1.000    0.067   48.400
    66.414     43.877    1.000    0.977   48.300
    43.877     66.414    0.627    0.204   48.200
    66.414     68.638    0.955    0.244   48.100
    68.63

In [75]:
ifig=12;close(ifig);fig=figure(ifig)
size=8
fig.canvas.layout.height = str(size)+'in'   # This is a hack to prevent ipympl
fig.canvas.layout.width  = str(size)+'in'   # to adjust horizontal figure size

plot(sini.T[0],sini.T[1],'s-',label='sini')
plot(sf.T[0],sf.T[1],'o--',label='sfinal')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …